In [1]:
from CompareSets import CompareSets
from CompareSignatures import CompareSignatures
from DocumentCollection import DocumentCollection
from MinHashing import MinHashing
from Shingling import Shingling
import csv
import random
import time

In [2]:
#get list from csv file
with open('usanews.csv','r', encoding='UTF-8', newline='') as f:
    reader = csv.reader(f)
    dicts = list(reader)

In [3]:
len(dicts)

10

In [4]:
k = 2 #Decide the length of shingles
x = 5 #Decide the number of documents for test
n = 100 # Decide the length of MinHash vector
s = 0.75 #Decide the threshold of Jaccard Similarity
bandwidth = 10 #Decide the threshold of LSH test

In [5]:
#Convert list above to an array
def getX(x):
    data_set = ['']*x
    for i in range(x):
        data_set[i] = ','.join('%s' %id for id in dicts[i])
    return data_set

#get random values for Minhashing Class
def getRand(n):
    A = []
    B = []
    P = []
    for i in range(n):
        A.append(random.randrange(3, 2000))
        B.append(random.randrange(3, 2000))
        P.append(random.randrange(3, 2000))
    return A, B, P

Values = getRand(n)

def getShinglets(documentA, documentB):
    #STEP 1 Shingling
    shinglingA = Shingling(k, documentA).construct_shinglets()
    shinglingB = Shingling(k, documentB).construct_shinglets()
    return shinglingA, shinglingB

def calculateSimilarityJaccard(documentA, documentB):
    #STEP 1 Shingling
    shinglingA, shinglingB = getShinglets(documentA, documentB)
    #STEP 2 Compare Shingling A and B
    return CompareSets(shinglingA, shinglingB).compareHashedShingles()

def calculateSimilarityMinHash(documentA, documentB):
    shinglingA, shinglingB = getShinglets(documentA, documentB)
    minHashSignatureA = MinHashing(Values[0], Values[1], shinglingA, Values[2]).getMinHashSignature(Values[0])
    minHashSignatureB = MinHashing(Values[0], Values[1], shinglingB, Values[2]).getMinHashSignature(Values[0])
    return CompareSignatures(minHashSignatureA, minHashSignatureB).getMinHashSimilarity()

def LSH(documentA, documentB, bandwidth):
    shinglingA, shinglingB = getShinglets(documentA, documentB)
    minHashSignatureA = MinHashing(Values[0], Values[1], shinglingA, Values[2]).getMinHashSignature(Values[0])
    minHashSignatureB = MinHashing(Values[0], Values[1], shinglingB, Values[2]).getMinHashSignature(Values[0])
    for i in range(int(len(minHashSignatureA)/bandwidth)):
        if (minHashSignatureA[i*bandwidth:(i+1)*bandwidth] == minHashSignatureB[i*bandwidth:(i+1)*bandwidth]):
           return True
    return False 

def main():
    #Load Data and Process it
    #Position in the List determines document ID
    start_time = time.time()
    documents = DocumentCollection(getX(x)).getDocuments(getX(x))
    similarities = []
    for i in range(len(documents)):
        for j in range(i + 1 , len(documents)):
            if(i == j):
                continue
            simJaccard = calculateSimilarityJaccard(documents[i], documents[j])
            simMinHash = calculateSimilarityMinHash(documents[i], documents[j])
            LSHtest = LSH(documents[i], documents[j], bandwidth)
            similarities.append((i, j, simJaccard, simMinHash, LSHtest))
    
    print("--- %s seconds ---" % (time.time() - start_time))
    
    #Print results
    print('Similarity List')
    for i in range(len(similarities)):
        print('docID:{}, docID:{}, Jaccard Similarity:{}. MinHash Signature Similarity {}. LSH: {}'.format(similarities[i][0], similarities[i][1], similarities[i][2], similarities[i][3], similarities[i][4]))
    
    #Print Similar Pairs
    print('Similar Pairs')
    for i in range(len(similarities)):
        if similarities[i][2] > s and similarities[i][4] is True:
            print('docID:{}, docID:{}'.format(similarities[i][0], similarities[i][1]))
    
    
if __name__ == "__main__":
    main()

--- 0.6220242977142334 seconds ---
Similarity List
docID:0, docID:1, Jaccard Similarity:0.053. MinHash Signature Similarity 0.41. LSH: False
docID:0, docID:2, Jaccard Similarity:0.121. MinHash Signature Similarity 0.42. LSH: False
docID:0, docID:3, Jaccard Similarity:0.011. MinHash Signature Similarity 0.34. LSH: False
docID:0, docID:4, Jaccard Similarity:0.04. MinHash Signature Similarity 0.37. LSH: False
docID:1, docID:2, Jaccard Similarity:0.072. MinHash Signature Similarity 0.44. LSH: False
docID:1, docID:3, Jaccard Similarity:0.006. MinHash Signature Similarity 0.38. LSH: False
docID:1, docID:4, Jaccard Similarity:0.024. MinHash Signature Similarity 0.32. LSH: False
docID:2, docID:3, Jaccard Similarity:0.011. MinHash Signature Similarity 0.35. LSH: False
docID:2, docID:4, Jaccard Similarity:0.029. MinHash Signature Similarity 0.37. LSH: False
docID:3, docID:4, Jaccard Similarity:0.013. MinHash Signature Similarity 0.39. LSH: False
Similar Pairs


In [8]:
k = 2
x = 10
n = 100
s = 0.75
bandwidth = 25
Values = getRand(n)#Update the random value for MinHashing
main()

--- 3.057786703109741 seconds ---
Similarity List
docID:0, docID:1, Jaccard Similarity:0.053. MinHash Signature Similarity 0.39. LSH: False
docID:0, docID:2, Jaccard Similarity:0.121. MinHash Signature Similarity 0.45. LSH: False
docID:0, docID:3, Jaccard Similarity:0.011. MinHash Signature Similarity 0.49. LSH: False
docID:0, docID:4, Jaccard Similarity:0.04. MinHash Signature Similarity 0.36. LSH: False
docID:0, docID:5, Jaccard Similarity:0.812. MinHash Signature Similarity 0.88. LSH: True
docID:0, docID:6, Jaccard Similarity:0.839. MinHash Signature Similarity 0.91. LSH: False
docID:0, docID:7, Jaccard Similarity:0.905. MinHash Signature Similarity 0.91. LSH: False
docID:0, docID:8, Jaccard Similarity:0.02. MinHash Signature Similarity 0.45. LSH: False
docID:0, docID:9, Jaccard Similarity:0.112. MinHash Signature Similarity 0.45. LSH: False
docID:1, docID:2, Jaccard Similarity:0.072. MinHash Signature Similarity 0.48. LSH: False
docID:1, docID:3, Jaccard Similarity:0.006. MinHash S